In [0]:
import argparse
import yaml
import time
import datetime
import cv2
import torch
from torch.autograd import Variable
import numpy as np
from astropy.visualization import make_lupton_rgb
import random
import numpy as np
from scipy.ndimage import zoom
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from scipy import ndimage, misc
import warnings
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import sys


In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r "/content/drive/My Drive/Colab/Mosquito_Berkeley/satellite" ./satellite

In [0]:
!ls satellite/

38.820636,-77.01538.png  38.898014,-76.97927.png  38.938009,-76.98513.png
38.831034,-77.00539.png  38.906416,-76.94851.png  38.939464,-77.05280.png
38.864641,-76.98592.png  38.906416,-76.98451.png  38.945614,-77.01564.png
38.8731,-76.97281.png	 38.907205,-77.05396.png  38.952145,-77.07169.png
38.874409,-76.95854.png  38.907725,-77.04406.png  38.954471,-77.06362.png
38.874658,-76.98730.png  38.913193,-76.99032.png  38.95682,-77.05096.png
38.875372,-77.03315.png  38.914004,-77.05731.png  38.956854,-77.05108.png
38.884384,-76.93195.png  38.914656,-77.09722.png  38.971088,-77.02990.png
38.887646,-77.04767.png  38.920896,-76.96704.png  38.972853,-77.05300.png
38.887771,-77.04232.png  38.921453,-77.01448.png  38.972854,-77.05297.png
38.888979,-77.00092.png  38.927132,-77.04890.png  38.980315,-77.05194.png
38.890913,-77.01556.png  38.931573,-77.04472.png  38.985287,-77.03816.png
38.891675,-77.01952.png  38.937768,-77.09681.png  satellite
38.891721,-77.02219.png  38.938009,-76.95813.png


In [0]:
!nvidia-smi

Wed Apr  8 18:26:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT','TRAPSET','TRAPCOLLECT']

In [0]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/original_mosquito.csv')
sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"]).dt.week
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"]).dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

# for col in numerical_columns:
#     sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())


In [0]:
def paddedzoom(img, zoomfactor=0.8):
    out  = np.zeros_like(img)
    zoomed = cv2.resize(img, None, fx=zoomfactor, fy=zoomfactor)
    
    h, w, _ = img.shape
    zh, zw, _ = zoomed.shape
    if zoomfactor<1:    
        out[int((h-zh)/2):int(-(h-zh)/2),int((w-zw)/2):int(-(w-zw)/2),:] = zoomed
    else:
        out = zoomed[int((zh-h)/2):int(-(zh-h)/2), int((zw-w)/2):int(-(zw-w)/2), :]

    return out

def get_random_crop(image, crop_height, crop_width):

    original_shape = image.shape
    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]
    resized = cv2.resize(crop, (416, 416), interpolation = cv2.INTER_AREA)

    return resized

def get_affeine(img, rows, cols):
    offset1 = ((random.random() * 2 ) - 1) * 40
    offset2 = ((random.random() * 2 ) - 1) * 40
    M = np.float32([[1,0,offset1],[0,1,offset2]])
    dst = cv2.warpAffine(img,M,(cols,rows))

    return dst

def distort(img, orientation='horizontal', func=np.sin, x_scale=0.05, y_scale=5):
    assert orientation[:3] in ['hor', 'ver'], "dist_orient should be 'horizontal'|'vertical'"
    assert func in [np.sin, np.cos], "supported functions are np.sin and np.cos"
    assert 0.00 <= x_scale <= 0.1, "x_scale should be in [0.0, 0.1]"
    assert 0 <= y_scale <= min(img.shape[0], img.shape[1]), "y_scale should be less then image size"
    img_dist = img.copy()
    
    def shift(x):
        return int(y_scale * func(np.pi * x * x_scale))
    
    for c in range(3):
        for i in range(img.shape[orientation.startswith('ver')]):
            if orientation.startswith('ver'):
                img_dist[:, i, c] = np.roll(img[:, i, c], shift(i))
            else:
                img_dist[i, :, c] = np.roll(img[i, :, c], shift(i))
            
    return img_dist

class SentinelDataset(Dataset):

    def __init__(self, root_dir, data, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        

        filename = str(self.data.iloc[idx, 6])[0:9]+',' + str(self.data.iloc[idx, 7])[0:9] + '.png'
        img_name = os.path.join(self.root_dir, filename)
        
        image = cv2.imread(img_name)
        image = cv2.resize(image, (416, 416), interpolation = cv2.INTER_AREA)

        #random flip
        if np.random.rand() > 0.5:
            image = np.flip(image, axis=1).copy()
        
        # if np.random.rand() > 0.5:
        #     image = distort(image, orientation=random.choice(['ver','hor']), x_scale=random.uniform(0.01, 0.03), y_scale=random.randint(2,10))
        
        #random rotate
        image = ndimage.rotate(image, random.randint(0,359), axes = [0,1],reshape=False)

        # random zoom
        or_image = image.copy()
        image = paddedzoom(image, 1.0 + (np.random.rand()/2) )
        if image.shape != or_image.shape:
            image = or_image
        
        if np.random.rand() > 0.2:
            image = get_random_crop(image, 380, 380)
        
        if np.random.rand() > 0.5:
            image = get_affeine(image, 416, 416)

        target = self.data.iloc[idx, -1]
        image = torch.from_numpy(image)


        return image, target

In [0]:
import torch
import torch.nn as nn

from collections import defaultdict

class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

def add_conv(in_ch, out_ch, ksize, stride):
    stage = nn.Sequential()
    pad = (ksize - 1) // 2
    stage.add_module('conv', nn.Conv2d(in_channels=in_ch,
                                       out_channels=out_ch, kernel_size=ksize, stride=stride,
                                       padding=pad, bias=False))
    stage.add_module('batch_norm', nn.BatchNorm2d(out_ch))
    stage.add_module('leaky', nn.LeakyReLU(0.1))
    return stage

def add_linear(in_ch, in_dm, out_ch, leaky = True):
    stage = nn.Sequential()
    stage.add_module('linear', nn.Linear(in_features=in_ch*in_dm*in_dm, out_features=out_ch))
    stage.add_module('batch_norm',nn.BatchNorm1d(num_features=out_ch))
    if leaky :
        stage.add_module('sigmoid', nn.Sigmoid())
    else:
        stage.add_module('relu', nn.ReLU())

    return stage


class resblock(nn.Module):
    def __init__(self, ch, nblocks=1, shortcut=True):

        super().__init__()
        self.shortcut = shortcut
        self.module_list = nn.ModuleList()
        for i in range(nblocks):
            resblock_one = nn.ModuleList()
            resblock_one.append(add_conv(ch, ch//2, 1, 1))
            resblock_one.append(add_conv(ch//2, ch, 3, 1))
            self.module_list.append(resblock_one)

    def forward(self, x):
        for module in self.module_list:
            h = x
            for res in module:
                h = res(h)
            x = x + h if self.shortcut else h
        return x

def create_darknet_modules():

    # DarkNet53
    mlist = nn.ModuleList()
    mlist.append(add_conv(in_ch=3, out_ch=32, ksize=3, stride=1))
    mlist.append(add_conv(in_ch=32, out_ch=64, ksize=3, stride=2))
    mlist.append(resblock(ch=64))
    mlist.append(add_conv(in_ch=64, out_ch=128, ksize=3, stride=2))
    mlist.append(resblock(ch=128, nblocks=2))
    mlist.append(add_conv(in_ch=128, out_ch=256, ksize=3, stride=2))
    mlist.append(resblock(ch=256, nblocks=8))
    mlist.append(add_conv(in_ch=256, out_ch=512, ksize=3, stride=2))
    mlist.append(resblock(ch=512, nblocks=8)) 
    mlist.append(add_conv(in_ch=512, out_ch=1024, ksize=3, stride=2))
    mlist.append(resblock(ch=1024, nblocks=4))

    mlist.append(Flatten())
    mlist.append(add_linear(in_ch=1024, in_dm=13, out_ch=1024))
    mlist.append(add_linear(in_ch=1024, in_dm=1, out_ch=1024))
    mlist.append(add_linear(in_ch=1024, in_dm=1, out_ch=1, leaky = False))

    return mlist

# def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout = 0.3):
#     all_layers = []
#     num_categorical_cols = sum((nf for ni, nf in embedding_size))
#     input_size = num_categorical_cols + num_numerical_cols
#     all_layers.append(nn.Linear(input_size, hidden_size))
#     all_layers.append(nn.Sigmoid())
#     all_layers.append(nn.Dropout(dropout))

#     all_layers.append(nn.Linear(hidden_size, hidden_size))
#     all_layers.append(nn.Sigmoid())
#     all_layers.append(nn.Dropout(dropout))

#     all_layers.append(nn.Linear(hidden_size, hidden_size))
#     all_layers.append(nn.Sigmoid())
#     all_layers.append(nn.Dropout(dropout))

#     all_layers.append(nn.Linear(hidden_size, output_size))
#     all_layers.append(nn.ReLU())

#     return nn.Sequential(*all_layers)

# def create_mix_modules(input_size, output_size, dropout = 0.3):
#     all_layers = []
#     all_layers.append(nn.Linear(input_size, input_size))
#     all_layers.append(nn.Sigmoid())
#     all_layers.append(nn.Dropout(dropout))

#     all_layers.append(nn.Linear(input_size, input_size))
#     all_layers.append(nn.Sigmoid())
#     all_layers.append(nn.Dropout(dropout))

#     all_layers.append(nn.Linear(input_size, output_size))
#     all_layers.append(nn.ReLU())
#     return nn.Sequential(*all_layers)

def turn_conv_lstm_autograd(self):

    def dfs_off(model):
        for name, child in model.named_children():
            for param in child.parameters():
                param.requires_grad = False
            dfs_off(child)

    for i, module in enumerate(self.module_list):
        if i == 11:
          break
        dfs_off(module)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size):
        super(Sentinel_net, self).__init__()
        self.module_list = create_darknet_modules()
        turn_conv_lstm_autograd(self)
    
        #self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, 65)
        #self.mix_layers = create_mix_modules(130, 1)
        #self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        #self.embedding_dropout = nn.Dropout(0.3)

    def forward(self, x, x_categorical, x_numerical):
        for i, module in enumerate(self.module_list):
            x = module(x)

        x = x.squeeze(dim=0)
        # embeddings = []
        # for i, e in enumerate(self.all_embeddings):
        #     embeddings.append(e(x_categorical[:, i]))
        

        # y = torch.cat(embeddings, 1)

        # y = self.embedding_dropout(y)
        # y = torch.cat([y, x_numerical], 1)
        # y = self.layers(y).squeeze()
        # mixtensor = torch.cat((x, y), 0)
        # mixtensor = self.mix_layers(mixtensor)
        return x

In [0]:
from __future__ import division
import torch
import numpy as np


def parse_conv_block(m, weights, offset, initflag):
    """
    Initialization of conv layers with batchnorm
    Args:
        m (Sequential): sequence of layers
        weights (numpy.ndarray): pretrained weights data
        offset (int): current position in the weights file
        initflag (bool): if True, the layers are not covered by the weights file. \
            They are initialized using darknet-style initialization.
    Returns:
        offset (int): current position in the weights file
        weights (numpy.ndarray): pretrained weights data
    """
    conv_model = m[0]
    bn_model = m[1]
    param_length = m[1].bias.numel()

    # batchnorm
    for pname in ['bias', 'weight', 'running_mean', 'running_var']:
        layerparam = getattr(bn_model, pname)

        if initflag: # yolo initialization - scale to one, bias to zero
            if pname == 'weight':
                weights = np.append(weights, np.ones(param_length))
            else:
                weights = np.append(weights, np.zeros(param_length))

        param = torch.from_numpy(weights[offset:offset + param_length]).view_as(layerparam)
        layerparam.data.copy_(param)
        offset += param_length

    param_length = conv_model.weight.numel()

    # conv
    if initflag: # yolo initialization
        n, c, k, _ = conv_model.weight.shape
        scale = np.sqrt(2 / (k * k * c))
        weights = np.append(weights, scale * np.random.normal(size=param_length))

    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.weight)
    conv_model.weight.data.copy_(param)
    offset += param_length

    return offset, weights

def parse_yolo_block(m, weights, offset, initflag):
    """
    YOLO Layer (one conv with bias) Initialization
    Args:
        m (Sequential): sequence of layers
        weights (numpy.ndarray): pretrained weights data
        offset (int): current position in the weights file
        initflag (bool): if True, the layers are not covered by the weights file. \
            They are initialized using darknet-style initialization.
    Returns:
        offset (int): current position in the weights file
        weights (numpy.ndarray): pretrained weights data
    """
    conv_model = m._modules['conv']
    param_length = conv_model.bias.numel()

    if initflag: # yolo initialization - bias to zero
        weights = np.append(weights, np.zeros(param_length))

    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.bias)
    conv_model.bias.data.copy_(param)
    offset += param_length

    param_length = conv_model.weight.numel()

    if initflag: # yolo initialization
        n, c, k, _ = conv_model.weight.shape
        scale = np.sqrt(2 / (k * k * c))
        weights = np.append(weights, scale * np.random.normal(size=param_length))
 
    param = torch.from_numpy(
        weights[offset:offset + param_length]).view_as(conv_model.weight)
    conv_model.weight.data.copy_(param)
    offset += param_length

    return offset, weights

def parse_yolo_weights(model, weights_path):
    """
    Parse YOLO (darknet) pre-trained weights data onto the pytorch model
    Args:
        model : pytorch model object
        weights_path (str): path to the YOLO (darknet) pre-trained weights file
    """
    fp = open(weights_path, "rb")

    # skip the header
    header = np.fromfile(fp, dtype=np.int32, count=5) # not used
    # read weights 
    weights = np.fromfile(fp, dtype=np.float32)
    fp.close()

    offset = 0 
    initflag = False #whole yolo weights : False, darknet weights : True

    for m in model.module_list:
        if m._get_name() == 'Sequential':
            # normal conv block
            offset, weights = parse_conv_block(m, weights, offset, initflag)

        elif m._get_name() == 'resblock':
            # residual block
            for modu in m._modules['module_list']:
                for blk in modu:
                    offset, weights = parse_conv_block(blk, weights, offset, initflag)
        else:
            break
        # elif m._get_name() == 'YOLOLayer':
        #     # YOLO Layer (one conv with bias) Initialization
        #     offset, weights = parse_yolo_block(m, weights, offset, initflag)

        # initflag = (offset >= len(weights)) # the end of the weights file. turn the flag on

In [0]:
def train_darknet_net(sentinel_dataset_train,sentinel_dataset_test):
    model = Sentinel_net(categorical_embedding_sizes,len(numerical_columns),1)
    parse_yolo_weights(model, "/content/drive/My Drive/Colab/mosquito/darknet53.conv.74")
    model = model.cuda()
    model.train()

    criterion = torch.nn.MSELoss(size_average = True) 

    params_dict = dict(model.named_parameters())
    params = []
    base_lr = 0.001

    for key, value in params_dict.items():
        if value.requires_grad:
          print(key)
          if 'weight' in key:
              params += [{'params':value, 'weight_decay':4e-3}]
          else:
              params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=base_lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)

    epochs = 100
    min_eval = sys.maxsize
    current_score = 0

    for epoch in range(0,epochs):
        loss_sum = 0
        
        model.train()
        for i_batch, sample_batched in enumerate(train_dataset_loader):
            image = sample_batched[0].cuda().float()
            image = image.permute(0,3,2,1)/255
            y_pred = model(image, None, None).squeeze()
            y = sample_batched[1].cuda().float()

            loss = criterion(y_pred, y) 
            loss.backward() 
            loss_sum += loss.item()
            optimizer.step() 
            optimizer.zero_grad() 

        model.eval()
        print('Train Accuracy epoch {}, loss {} %'.format(epoch, loss_sum))

        with torch.no_grad():

            if epoch % 10 == 0:
                model.eval()
                eval_loss_sum = 0 
                result_pred = []
                result_act = []
                with torch.no_grad():
                    y2 = None
                    y_pred2 = None
                    for i_batch2, sample_batched2 in enumerate(test_dataset_loader):

                        image2 = sample_batched2[0].cuda().float()
                        image2 = image2.permute(0,3,2,1)/255
                        y_pred2 = model(image2,None,None).squeeze()
                        y2 = sample_batched2[1].cuda().float()
                        eval_loss_sum += criterion(y_pred2, y2) 
                        result_pred += y_pred2.tolist()
                        result_act += y2.tolist()
                    
                    print('')
                    print(' Test Accuracy  {} %'.format( eval_loss_sum))


                    if eval_loss_sum < min_eval:
                        current_score = mean_absolute_error(result_pred, result_act)
                        print("Score!" ,current_score )
                        min_eval = eval_loss_sum
                        print("saved!" ,current_score )
                        torch.save({'iter': epoch,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'best_ac': min_eval,
                          },
                          os.path.join('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/best_approach' + str(fold_count) + '.ckpt'))


    return current_score

In [0]:
scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=True)
fold_count = 0
for train_index, test_index in cv.split(sentinel_frame, None):
    print('Fold')
    train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

    sentinel_dataset_train = SentinelDataset(data=train, root_dir='satellite/')
    sentinel_dataset_test = SentinelDataset(data=test, root_dir='satellite/')

    train_dataset_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset_train,
                                                    batch_size=20,
                                                    shuffle=False)
    
    test_dataset_loader = torch.utils.data.DataLoader(dataset=sentinel_dataset_test,
                                                    batch_size=20,
                                                    shuffle=False)
    

    current_score = train_darknet_net(sentinel_dataset_train,sentinel_dataset_test)
    scores.append(current_score)
    fold_count += 1
print(scores)
print( sum(scores) / len(scores) )


Fold


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
module_list.14.linear.weight
module_list.14.linear.bias
module_list.14.batch_norm.weight
module_list.14.batch_norm.bias
Train Accuracy epoch 0, loss 96039.78586578369 %

 Test Accuracy  15520.3388671875 %
Score! 12.944622122091644
saved! 12.944622122091644
Train Accuracy epoch 1, loss 93791.61571598053 %
Train Accuracy epoch 2, loss 92746.08248806 %
Train Accuracy epoch 3, loss 91366.61149215698 %
Train Accuracy epoch 4, loss 90732.4849948883 %
Train Accuracy epoch 5, loss 89873.07434463501 %
Train Accuracy epoch 6, loss 88388.02087306976 %
Train Accuracy epoch 7, loss 88130.46438789368 %
Train Accuracy epoch 8, loss 88287.86022853851 %
Train Accuracy epoch 9, loss 87217.29424762726 %
Train Accuracy epoch 10, loss 86318.55828380585 %

 Test Accuracy 

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
module_list.14.linear.weight
module_list.14.linear.bias
module_list.14.batch_norm.weight
module_list.14.batch_norm.bias
Train Accuracy epoch 0, loss 88009.30087280273 %

 Test Accuracy  23080.470703125 %
Score! 14.496053413292508
saved! 14.496053413292508
Train Accuracy epoch 1, loss 86284.99170398712 %
Train Accuracy epoch 2, loss 85035.01092910767 %
Train Accuracy epoch 3, loss 83678.39536476135 %
Train Accuracy epoch 4, loss 82497.32294845581 %
Train Accuracy epoch 5, loss 82916.06280517578 %
Train Accuracy epoch 6, loss 81806.07965660095 %
Train Accuracy epoch 7, loss 81158.93196678162 %
Train Accuracy epoch 8, loss 80042.79108810425 %
Train Accuracy epoch 9, loss 79517.92651557922 %
Train Accuracy epoch 10, loss 79393.70984649658 %

 Test Accura

...

module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
module_list.14.linear.weight
module_list.14.linear.bias
module_list.14.batch_norm.weight
module_list.14.batch_norm.bias
Train Accuracy epoch 0, loss 91381.3415260315 %

 Test Accuracy  20133.04296875 %
Score! 12.533837504696423
saved! 12.533837504696423
Train Accuracy epoch 1, loss 89617.3055934906 %
Train Accuracy epoch 2, loss 87901.91896247864 %
Train Accuracy epoch 3, loss 87172.88298225403 %
Train Accuracy epoch 4, loss 87030.78859138489 %
Train Accuracy epoch 5, loss 84784.20680999756 %
Train Accuracy epoch 6, loss 85153.33562088013 %
Train Accuracy epoch 7, loss 83494.57718467712 %
Train Accuracy epoch 8, loss 82503.27345466614 %
Train Accuracy epoch 9, loss 81733.48722553253 %
Train Accuracy epoch 10, loss 81917.75102424622 %

 Test Accuracy 

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
module_list.14.linear.weight
module_list.14.linear.bias
module_list.14.batch_norm.weight
module_list.14.batch_norm.bias
Train Accuracy epoch 0, loss 71887.04796218872 %

 Test Accuracy  39776.8203125 %
Score! 13.260631238906829
saved! 13.260631238906829
Train Accuracy epoch 1, loss 70308.46635627747 %
Train Accuracy epoch 2, loss 68633.20254898071 %
Train Accuracy epoch 3, loss 67301.97316169739 %
Train Accuracy epoch 4, loss 66497.79860019684 %
Train Accuracy epoch 5, loss 65552.62239074707 %
Train Accuracy epoch 6, loss 64819.076389312744 %
Train Accuracy epoch 7, loss 63623.23511505127 %
Train Accuracy epoch 8, loss 63307.466875076294 %
Train Accuracy epoch 9, loss 62133.093309402466 %
Train Accuracy epoch 10, loss 62219.15185165405 %

 Test Accur

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


module_list.12.linear.weight
module_list.12.linear.bias
module_list.12.batch_norm.weight
module_list.12.batch_norm.bias
module_list.13.linear.weight
module_list.13.linear.bias
module_list.13.batch_norm.weight
module_list.13.batch_norm.bias
module_list.14.linear.weight
module_list.14.linear.bias
module_list.14.batch_norm.weight
module_list.14.batch_norm.bias
Train Accuracy epoch 0, loss 99775.01598358154 %

 Test Accuracy  11654.37109375 %
Score! 11.913454268434567
saved! 11.913454268434567
Train Accuracy epoch 1, loss 97668.88698577881 %
Train Accuracy epoch 2, loss 96308.84237003326 %
Train Accuracy epoch 3, loss 94758.21427345276 %
Train Accuracy epoch 4, loss 94223.57270002365 %
Train Accuracy epoch 5, loss 93469.47418880463 %
Train Accuracy epoch 6, loss 92434.33746814728 %
Train Accuracy epoch 7, loss 92457.52846431732 %
Train Accuracy epoch 8, loss 91045.13549613953 %
Train Accuracy epoch 9, loss 91230.58498287201 %
Train Accuracy epoch 10, loss 90060.03569316864 %

 Test Accurac